# Heart Disease Prediction Guided Project

This is a guided project from Dataquest's course "Introduction to Supervised Machine Learning in Python".

According to the World Health Organisation (WHO), an estimated 17.9 million people die each year from cardiovascular diseases. There are manifold risk factors that potentially contribute to cardiovascular diseases in a person, such as an unhealthy diet, lack of physical activity, and mental illness.

The [dataset](https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction) in this project contains relevant information for each patient, including their personal information, some medical data, and whether they have had heart disease. The aim is to use this dataset to write a k-Nearest Neighbours machine learning model to accurately predict the likelihood of a new patient having heart disease in the future.